In [1]:
import pandas as pd
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
import numpy as np

In [2]:
animes = pd.read_csv("animes.csv")
user_test = pd.read_csv("user_rates_test.csv")
user = pd.read_csv("user_rates_train.csv")


In [3]:
animes.drop(["Image URL","Name"], axis=1,inplace=True)

In [4]:
user.drop("Username", axis=1,inplace=True)

In [5]:
user.columns

Index(['user_id', 'anime_id', 'Anime Title', 'rating'], dtype='object')

In [6]:
animes.columns

Index(['anime_id', 'Genres', 'Type', 'Studios', 'Source', 'Duration'], dtype='object')

In [7]:
df = pd.merge(animes, user, on='anime_id', how='inner')
df 

,anime_id,Genres,Type,Studios,Source,Duration,user_id,Anime Title,rating
0,1,"Action, Award Winning, Sci-Fi",TV,Sunrise,Original,24 min per ep,549,Cowboy Bebop,10
1,1,"Action, Award Winning, Sci-Fi",TV,Sunrise,Original,24 min per ep,644,Cowboy Bebop,10
2,1,"Action, Award Winning, Sci-Fi",TV,Sunrise,Original,24 min per ep,2341,Cowboy Bebop,9
3,1,"Action, Award Winning, Sci-Fi",TV,Sunrise,Original,24 min per ep,3242,Cowboy Bebop,8
4,1,"Action, Award Winning, Sci-Fi",TV,Sunrise,Original,24 min per ep,3544,Cowboy Bebop,8
...,...,...,...,...,...,...,...,...,...
49355,52305,"Comedy, Romance",TV,Lay-duce,Web manga,23 min per ep,64873,Tomo-chan wa Onnanoko!,9
49356,52865,"Comedy, Romance, Supernatural",ONA,domerica,Manga,26 min per ep,422316,Romantic Killer,6
49357,52865,"Comedy, Romance, Supernatural",ONA,domerica,Manga,26 min per ep,1064281,Romantic Killer,9
49358,53148,Comedy,ONA,J.C.Staff,Manga,17 min per ep,64873,Gokushufudou Season 2,9


# K fold yapılıcak

In [8]:
# Remove 'per ep' string from duration to make convertion simple
df ['Duration'] = df ['Duration'].str.replace('per ep', '', regex=False)


In [9]:
# Function to convert time strings to minutes
def convert_to_minutes(time_str):
    parts = time_str.split()
    total_minutes = 0
    for part in parts:
        if part.isdigit():
            total_minutes += int(part)
        elif part == 'hr' or part == 'hrs':
            total_minutes += int(parts[parts.index(part) - 1]) * 60
    return total_minutes

# Apply the conversion function to the 'time_column'
df ['Duration'] = df ['Duration'].apply(convert_to_minutes)


In [10]:

genres =list()
for i in df["Genres"]:
    tpl = list()
    i =i.replace(" ","") # It is necessary for splitting data correctly
    tpl.append(i.split(","))
  
    for j in tpl[0]:
        if not j in genres:
            genres.append(j)
print(genres)
print(len(genres))



['Action', 'AwardWinning', 'Sci-Fi', 'Adventure', 'Drama', 'Mystery', 'Supernatural', 'Fantasy', 'Sports', 'Comedy', 'Romance', 'SliceofLife', 'Suspense', 'Ecchi', 'Gourmet', 'AvantGarde', 'Horror', 'GirlsLove', 'BoysLove', 'Hentai', 'UNKNOWN', 'Erotica']
22


In [11]:
# Make floor division with ten. Because there is no such differences between users which watchs 22 minutes anime and 25 minute anime

def divide_by_10(minutes):
    return minutes // 10

# Apply the custom function to the 'Duration' column
df['Duration'] = df['Duration'].apply(divide_by_10)
df['Duration'] 

0        2
1        2
2        2
3        2
4        2
        ..
49355    2
49356    2
49357    2
49358    1
49359    2
Name: Duration, Length: 49360, dtype: int64

In [12]:
df['Genres'] = df['Genres'].str.split(', ')

In [13]:
average_rate = df.groupby('user_id')['rating'].mean().reset_index()

# Rename the columns if needed
average_rate.columns = ['user_id', 'rating']

# Create a new DataFrame with the results
user_points = average_rate

# Display the new DataFrame
user_points

,user_id,rating
0,549,8.400000
1,644,8.666667
2,1519,8.375000
3,2211,7.916667
4,2297,8.333333
...,...,...
2995,1289083,8.800000
2996,1289493,7.785714
2997,1289795,8.058824
2998,1289833,9.200000


In [14]:

# Calculate the maximum frequency of points for each unique user_id
max_frequency_points = df.groupby('user_id')['Duration'].value_counts().reset_index(name='Duration_max_frequency')

# Get the rows with the maximum frequency for each user
max_frequency_points = max_frequency_points.sort_values(by=['user_id', 'Duration_max_frequency'], ascending=[True, False]).drop_duplicates('user_id')

# Create a new DataFrame with the results
result_df = max_frequency_points
user_points = pd.merge(user_points, max_frequency_points, on='user_id', how='inner')

user_points

,user_id,rating,Duration,Duration_max_frequency
0,549,8.400000,2,17
1,644,8.666667,2,11
2,1519,8.375000,2,12
3,2211,7.916667,2,23
4,2297,8.333333,2,11
...,...,...,...,...
2995,1289083,8.800000,2,17
2996,1289493,7.785714,2,10
2997,1289795,8.058824,2,13
2998,1289833,9.200000,2,20


In [15]:

# Calculate the maximum frequency of points for each unique user_id
max_frequency_points = df.groupby('user_id')['Studios'].value_counts().reset_index(name='Studios_max_frequency')

# Get the rows with the maximum frequency for each user
max_frequency_points = max_frequency_points.sort_values(by=['user_id', 'Studios_max_frequency'], ascending=[True, False]).drop_duplicates('user_id')

# Create a new DataFrame with the results
result_df = max_frequency_points
user_points = pd.merge(user_points, max_frequency_points, on='user_id', how='inner')
user_points


,user_id,rating,Duration,Duration_max_frequency,Studios,Studios_max_frequency
0,549,8.400000,2,17,Madhouse,4
1,644,8.666667,2,11,Xebec,5
2,1519,8.375000,2,12,Gonzo,3
3,2211,7.916667,2,23,Pierrot,5
4,2297,8.333333,2,11,Studio Deen,3
...,...,...,...,...,...,...
2995,1289083,8.800000,2,17,Production I.G,6
2996,1289493,7.785714,2,10,E&G Films,3
2997,1289795,8.058824,2,13,J.C.Staff,3
2998,1289833,9.200000,2,20,AIC ASTA,2


In [16]:

# Calculate the maximum frequency of points for each unique user_id
max_frequency_points = df.groupby('user_id')['Source'].value_counts().reset_index(name='Source_max_frequency')

# Get the rows with the maximum frequency for each user
max_frequency_points = max_frequency_points.sort_values(by=['user_id', 'Source_max_frequency'], ascending=[True, False]).drop_duplicates('user_id')

# Create a new DataFrame with the results
result_df = max_frequency_points
user_points = pd.merge(user_points, max_frequency_points, on='user_id', how='inner')
user_points


,user_id,rating,Duration,Duration_max_frequency,Studios,Studios_max_frequency,Source,Source_max_frequency
0,549,8.400000,2,17,Madhouse,4,Manga,8
1,644,8.666667,2,11,Xebec,5,Manga,9
2,1519,8.375000,2,12,Gonzo,3,Manga,6
3,2211,7.916667,2,23,Pierrot,5,Manga,12
4,2297,8.333333,2,11,Studio Deen,3,Manga,10
...,...,...,...,...,...,...,...,...
2995,1289083,8.800000,2,17,Production I.G,6,Manga,19
2996,1289493,7.785714,2,10,E&G Films,3,Light novel,5
2997,1289795,8.058824,2,13,J.C.Staff,3,Manga,10
2998,1289833,9.200000,2,20,AIC ASTA,2,Manga,11


In [17]:
user_points

,user_id,rating,Duration,Duration_max_frequency,Studios,Studios_max_frequency,Source,Source_max_frequency
0,549,8.400000,2,17,Madhouse,4,Manga,8
1,644,8.666667,2,11,Xebec,5,Manga,9
2,1519,8.375000,2,12,Gonzo,3,Manga,6
3,2211,7.916667,2,23,Pierrot,5,Manga,12
4,2297,8.333333,2,11,Studio Deen,3,Manga,10
...,...,...,...,...,...,...,...,...
2995,1289083,8.800000,2,17,Production I.G,6,Manga,19
2996,1289493,7.785714,2,10,E&G Films,3,Light novel,5
2997,1289795,8.058824,2,13,J.C.Staff,3,Manga,10
2998,1289833,9.200000,2,20,AIC ASTA,2,Manga,11


In [18]:
from collections import Counter
concatenated_genres = df.groupby('user_id')['Genres'].sum().reset_index()

def most_repeated_element(lst):
    if not lst:
        return None  # Return None for empty lists
    element_count = Counter(lst)
    most_common_element = element_count.most_common(1)
    return most_common_element[0][0]

# Apply the function to the 'Genres' column and replace the lists with most repeated elements
concatenated_genres['Genres'] =concatenated_genres['Genres'].apply(most_repeated_element)

In [19]:
user_total = pd.merge(user_points, concatenated_genres, on='user_id', how='inner')


In [20]:
user_total = user_total.rename(columns={'Duration': 'Max_Duration'})
user_total = user_total.rename(columns={'Studios': 'Max_Studios'})
user_total = user_total.rename(columns={'Source': 'Max_Source'})
user_total

,user_id,rating,Max_Duration,Duration_max_frequency,Max_Studios,Studios_max_frequency,Max_Source,Source_max_frequency,Genres
0,549,8.400000,2,17,Madhouse,4,Manga,8,Action
1,644,8.666667,2,11,Xebec,5,Manga,9,Comedy
2,1519,8.375000,2,12,Gonzo,3,Manga,6,Comedy
3,2211,7.916667,2,23,Pierrot,5,Manga,12,Action
4,2297,8.333333,2,11,Studio Deen,3,Manga,10,Comedy
...,...,...,...,...,...,...,...,...,...
2995,1289083,8.800000,2,17,Production I.G,6,Manga,19,Action
2996,1289493,7.785714,2,10,E&G Films,3,Light novel,5,Fantasy
2997,1289795,8.058824,2,13,J.C.Staff,3,Manga,10,Romance
2998,1289833,9.200000,2,20,AIC ASTA,2,Manga,11,Comedy


In [21]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Apply label encoding to the 'Category' column
user_total['Max_Studios'] = label_encoder.fit_transform(user_total['Max_Studios'])
user_total['Max_Source'] = label_encoder.fit_transform(user_total['Max_Source'])
user_total['Genres'] = label_encoder.fit_transform(user_total['Genres'])

user_total

,user_id,rating,Max_Duration,Duration_max_frequency,Max_Studios,Studios_max_frequency,Max_Source,Source_max_frequency,Genres
0,549,8.400000,2,17,43,4,3,8,0
1,644,8.666667,2,11,78,5,3,9,4
2,1519,8.375000,2,12,33,3,3,6,4
3,2211,7.916667,2,23,50,5,3,12,0
4,2297,8.333333,2,11,62,3,3,10,4
...,...,...,...,...,...,...,...,...,...
2995,1289083,8.800000,2,17,53,6,3,19,0
2996,1289493,7.785714,2,10,27,3,2,5,7
2997,1289795,8.058824,2,13,37,3,3,10,12
2998,1289833,9.200000,2,20,5,2,3,11,4


In [22]:
user_y = user_total["rating"]
user_x = user_total.drop("rating",axis = 1)
user_x = user_x.drop("Duration_max_frequency",axis = 1)
user_x = user_x.drop("Studios_max_frequency",axis = 1) 
user_x = user_x.drop("Source_max_frequency",axis = 1)

In [23]:
user_x

,user_id,Max_Duration,Max_Studios,Max_Source,Genres
0,549,2,43,3,0
1,644,2,78,3,4
2,1519,2,33,3,4
3,2211,2,50,3,0
4,2297,2,62,3,4
...,...,...,...,...,...
2995,1289083,2,53,3,0
2996,1289493,2,27,2,7
2997,1289795,2,37,3,12
2998,1289833,2,5,3,4


# Cosine Similarity

In [24]:
def knnSimilarity(user_id,k_number):
    correlations = list()
    user_ids = list()



    vector1 = user_total.iloc[user_id]
    vector1 = vector1.drop("user_id")

    for i in range(len(user_total)):

        vector2 = user_total.iloc[i]
        user_ids.append(vector2["user_id"])

        vector2= vector2.drop("user_id")

        dot_product = np.dot(vector1 , vector2)
        magnitude1 = np.linalg.norm(vector1)
        magnitude2 = np.linalg.norm(vector2)

        # Calculate the cosine similarity
        cosine_similarity = dot_product / (magnitude1 * magnitude2)
        correlations.append(cosine_similarity)
        my_dict = dict(zip(user_ids, correlations))


    # Sort the list in ascending order
    sorted_dict = dict(sorted(my_dict.items(), key=lambda item: item[1]))

    # Convert the dictionary to a list of key-value pairs
    items = list(sorted_dict.items())

    # Get the last three items
    last_three_items = items[-(k_number+1):-1]

    # Convert the last three items back to a dictionary
    result_dict = dict(last_three_items)

    return (result_dict)



In [25]:
id = 549
k = 3
knnSimilarity(id,k)

{299361.0: 0.99854649807702,
 1079441.0: 0.9989443201279413,
 47486.0: 0.9995629433400793}